In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
directory = '../input/chest_xray/chest_xray/train/'

choices = ['NORMAL' , 'PNEUMONIA']

training_data = []

for i in choices:
    path = directory + i
    for j in os.listdir(path):
        try:
            arr = cv2.imread(path + '/' +j , 0)
            new_arr = cv2.resize(arr , (28,28))
            training_data.append([new_arr , choices.index(i)])
        except Exception as e:
            pass
#print(training_data)

import random

random.shuffle(training_data)

X = []
Y = []
for f , l in training_data:
    X.append(f)
    Y.append(l)
X = np.array(X)
Y = np.array(Y)

In [ ]:
X = X / 225.0

X = X.reshape(-1 , 28 , 28, 1)

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(256 ,(3,3) , input_shape = (28 , 28 , 1)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

model.add(tf.keras.layers.Conv2D(256 ,(3,3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

model.add(tf.keras.layers.Conv2D(256 ,(3,3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['acc'])

In [ ]:
validation = []

for i in choices:
    path = "../input/chest_xray/chest_xray/val/" + i
    for j in os.listdir(path):
        try:
            arr = cv2.imread(path + '/' +j , 0)
            new_arr = cv2.resize(arr , (28,28))
            validation.append([new_arr , choices.index(i)])
        except Exception as e:
            pass
        
random.shuffle(validation)

Xv = []
Yv = []
for f , l in validation:
    Xv.append(f)
    Yv.append(l)
Xv = np.array(X)
Yv = np.array(Y)

In [ ]:
his = model.fit(X , Y , validation_data = (Xv , Yv) , epochs = 10)

In [ ]:
import matplotlib.pyplot as plt

print(his.history.keys())
# summarize history for loss
plt.plot(his.history['loss'])
plt.plot(his.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

np.argmin(his.history['val_loss'])

model.save_weights('weights.h5')

In [ ]:
from sklearn.metrics import accuracy_score

test_data = []
choice = ["NORMAL" , "PNEUMONIA"]
for i in choice:
    path = "../input/chest_xray/chest_xray/test/" + i
    for j in os.listdir(path):
        try:
            arr = cv2.imread(path + '/' +j , 0)
            new_arr = cv2.resize(arr , (28,28))
            test_data.append([new_arr , choice.index(i)])
        except Exception as e:
            pass
        
random.shuffle(test_data)

Xt = []
Yt = []
for f , l in test_data:
    Xt.append(f)
    Yt.append(l)
Xt = np.array(Xt)
Yt = np.array(Yt)

pred = model.predict(Xt.reshape(-1,28,28 , 1))
predictions = []
for i in pred:
    if i < 0.5:
        predictions.append(0)
    else:
        predictions.append(1)

accuracy_score(predictions , Yt)

In [ ]:
model.save_weights('weight.h5')